# Assignment 2

In [2]:

# Import necessary packages
import tarfile
import os
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import locale

In [12]:
file_path = "kungalv_slutpriser.tar.gz"

if file_path.endswith("tar.gz"):
    tar = tarfile.open(file_path, "r:gz")
    tar.extractall()
    tar.close()
elif file_path.endswith("tar"):
    tar = tarfile.open(file_path, "r:")
    tar.extractall()
    tar.close()



In [3]:
# Helper Functions
def clean_date(date):
    locale.setlocale(locale.LC_ALL, 'sv_SE')
    datetime_object = datetime.strptime(date, '%d %B %Y')
    return datetime_object.strftime("%Y-%m-%d")



In [52]:
import re 
with open("kungalv_slutpriser/kungalv_slutpris_page_01.html") as fp:
    soup = BeautifulSoup(fp, 'html.parser')

# date of sale
listings = soup.findAll('li', class_ = 'sold-results__normal-hit')

listing_elements = []
for listing in listings:
    listing_soup = BeautifulSoup(str(listing), 'html.parser')

    area = listing_soup.find('div', {'class' : 'sold-property-listing__subheading sold-property-listing__area'}).text.strip()
    # TO DO PRITISH: Works but needs regex implementation
    area = re.sub(r'\s', ' ', area.strip())
    arealist = re.findall(r'\d+', area)

    if len(arealist) == 3:
        boarea = int(arealist[0])
        biarea = int(arealist[1])
        rooms = int(arealist[2])
        livingarea = boarea + biarea
    elif len(arealist) == 2:
        livingarea = int(arealist[0])
        rooms = int(arealist[1])

    date = listing_soup.find('span', {'class' : 'hcl-label hcl-label--state hcl-label--sold-at'}).text.strip()
    match = re.search(r'\d', date)  # Search for the first number
    date_of_sale = clean_date((date[match.start():]))
    address = listing_soup.find('h2', {'class' : 'sold-property-listing__heading qa-selling-price-title hcl-card__title'}).text.strip()
    location = listing_soup.find('div', {'class' : 'sold-property-listing__location'}).text
    location = re.sub(r'\s+', ' ', location.split("VillaVilla")[1].strip())
    listingarea = listing_soup.find('div', {'class' : 'sold-property-listing__subheading sold-property-listing__area'}).text.strip()
    plotarea = listing_soup.find('div', {'class' : 'sold-property-listing__land-area'}).text.strip()
    closingprice = listing_soup.find('span',{'class' : 'hcl-text hcl-text--medium'}).text.strip()
   # print(area, dateofsale, address, location, listingarea, plotarea, closingprice)
    
    # Append the attributes for each listing to a list
    listing_info = [livingarea, rooms, date_of_sale, address, location, plotarea, closingprice]
    
    # Append the list of attributes to the overall list of listings
    listing_elements.append(listing_info)
print(*listing_elements,sep = "\n")



 

[168, 7, '2023-10-09', 'Skårby station 350', 'Kareby, Kungälvs kommun', '2\xa0303\xa0m² tomt', 'Slutpris 3\xa0005\xa0000\xa0kr']
[206, 5, '2023-10-05', 'Högalidsgatan 3', 'Centrum, Kungälvs kommun', '862\xa0m² tomt', 'Slutpris 3\xa0800\xa0000\xa0kr']
[123, 5, '2023-10-03', 'Kungälvsvägen 22', 'Centralt, Kungälvs kommun', '1\xa0548\xa0m² tomt', 'Slutpris 4\xa0500\xa0000\xa0kr']
[123, 6, '2023-10-02', 'Ädelstensvägen 58', 'Kode, Kungälvs kommun', '379\xa0m² tomt', 'Slutpris 4\xa0075\xa0000\xa0kr']
[166, 6, '2023-09-27', 'Kantorvägen 4', 'Bohuslän, Kungälvs kommun', '558\xa0m² tomt', 'Slutpris 3\xa0625\xa0000\xa0kr']
[123, 5, '2023-09-26', 'Diamantvägen 34', 'Kode, Kungälvs kommun', '559\xa0m² tomt', 'Slutpris 2\xa0900\xa0000\xa0kr']
[126, 4, '2023-09-25', 'Tjäderstigen 8', 'Centralt, Kungälvs kommun', '362\xa0m² tomt', 'Slutpris 4\xa0760\xa0000\xa0kr']
[195, 6, '2023-09-22', 'Heavägen 31', 'Lycke, Kungälvs kommun', '1\xa0553\xa0m² tomt', 'Slutpris 5\xa0450\xa0000\xa0kr']
[145, 5, '2023-0

In [ ]:
   

    
#address
    
#location
    
#area in the form (boarea + biarea)

#number of rooms
    
#area of the plot
    

#closing price
